In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20,10

from sklearn import linear_model

## Predictor Variables

In [2]:
# Commodity Prices
dfCommodity = pd.read_csv('../data/commodityPrices.csv')
dfCommodity['date'] = pd.to_datetime(dfCommodity['date'])
dfCommodity = dfCommodity.set_index('date').sort_index()

# Wind Generation
dfWind = pd.read_csv('../data/MISOWindGeneration.csv')
dfWind['date'] = pd.to_datetime(dfWind['date'])
dfWind = dfWind.set_index('date').sort_index()

# Demand
dfLoad = pd.read_csv('../data/MISOActualLoad.csv')
dfLoad['Market Day'] = pd.to_datetime(dfLoad['Market Day'])
dfLoad = dfLoad.set_index('Market Day').sort_index()
dfLoad.index.names = ['date']
dfLoadActual = dfLoad[['Central ActualLoad (MWh)', 'East ActualLoad (MWh)', 'MISO ActualLoad (MWh)', 'Midwest ISO ActualLoad (MWh)', 'North ActualLoad (MWh)', 'South ActualLoad (MWh)', 'West ActualLoad (MWh)']]
dfLoadActual = dfLoadActual.fillna(0)    # Handle NaN

In [3]:
# Merge into a single DataFrame
dfX = pd.merge(dfCommodity, dfWind, left_index=True, right_index=True)
dfX = pd.merge(dfX, dfLoadActual, left_index=True, right_index=True)
dfX.head()

,Central Appalachia,Northern Appalachia,Illinois Basin,Powder River Basin,Uinta Basin,NgPrice,windGenerationMWh,Central ActualLoad (MWh),East ActualLoad (MWh),MISO ActualLoad (MWh),Midwest ISO ActualLoad (MWh),North ActualLoad (MWh),South ActualLoad (MWh),West ActualLoad (MWh)
date,,,,,,,,,,,,,,
2009-07-06,53.333333,51.166667,44,9,44.5,3.355000,325.927083,21654.299167,28127.989583,0,62493.192500,0,0,12710.903750
2009-07-07,53.500000,52.000000,44,9,44.5,3.300000,511.056250,22956.574167,27699.694167,0,63508.234167,0,0,12851.965833
2009-07-08,53.500000,52.000000,44,9,44.5,3.298571,728.637083,22402.212500,27335.640833,0,62295.661250,0,0,12557.807917
2009-07-09,53.500000,52.000000,44,9,44.5,3.297143,1716.773333,23165.527500,28635.972917,0,64392.903333,0,0,12591.402917
2009-07-10,53.500000,52.000000,44,9,44.5,3.295714,837.547917,23676.907917,29816.782083,0,66701.584583,0,0,13207.894583


## Response Variable

In [4]:
dfMiso = pd.read_hdf('../data/LMP.h5')

## Data Prep before Regression

In [6]:
%%time

# Collapse MultiIndex of dfY(to prep for merge with dfX)
dfY = dfMiso.dropna()                                   # Drop rows with NA
dfY = dfY.reset_index()                                 # collapse MultiIndex
dfY = dfY[['date', 'meanPrice']].set_index('date')
dfY.index = pd.to_datetime(dfY.index, format='%Y%m%d')  # expensive operation (parsing 20,000,000 dates)

CPU times: user 24.8 s, sys: 3.78 s, total: 28.6 s
Wall time: 29.8 s


In [7]:
dfY.head()

,meanPrice
date,
2005-06-16,29.463750
2005-06-16,-0.947083
2005-06-16,-0.393333
2005-06-17,28.973750
2005-06-17,1.207917


In [8]:
# Inner Join on 20,000,000 rows! (3.45 secs)
df = pd.merge(dfY, dfX, left_index=True, right_index=True, how='inner')
df = df[:'2013-09-01']

# Plot (Do not plot this! Takes too long. Looks the same as the regular plots)
# df.plot()

## Random Forest Regression (All Nodes)
Get a feel of training time for 20 million data points

In [23]:
%%time

from sklearn.cross_validation import train_test_split
from sklearn import grid_search
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from scipy import stats


# Outlier Removal: Very low LMP
df2 = df.copy()
df2 = df2[df2['meanPrice'] > 15]


# Log Transform
df2['meanPrice'] = np.log(df2['meanPrice'] + 50)
df2['NgPrice'] = np.log(df2['NgPrice'])


# Log Transform - Translate by tiny amount to avoid np.log(0)
df2['Northern Appalachia'] = np.log(df2['Northern Appalachia'] + 0.1)
df2['Illinois Basin'] = np.log(df2['Illinois Basin'] + 0.1)
df2['Uinta Basin'] = np.log(df2['Uinta Basin'] + 0.1)
df2['Powder River Basin'] = np.log(df2['Uinta Basin'] + 0.1)
df2['Central Appalachia'] = np.log(df2['Central Appalachia'] + 0.1)


# Outlier Removal: 2-sd
df2 = df2[(np.abs(stats.zscore(df2['meanPrice'])) < 2)]

# Split into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(df2.drop('meanPrice', axis=1), df2['meanPrice'], 
                                                    test_size=0.2, random_state=0)


# Train model
rf = RandomForestRegressor(n_jobs=3, min_samples_split=5, n_estimators=75, max_depth=5)
fit = rf.fit(X_train, y_train)


# Metrics
preds = rf.predict(X_test)
print 'R2 Score: ', metrics.r2_score(y_test.values, preds)
print 'Explained Variance Score: ', metrics.explained_variance_score(y_test.values, preds)
print 'MAE: ', metrics.mean_absolute_error(y_test.values, preds)
print 'MSE: ', metrics.mean_squared_error(y_test.values, preds)
print 'Median AE: ', metrics.median_absolute_error(y_test.values, preds)
print

# Feature Importances
lcols = X_train.columns
pd.DataFrame(zip(lcols, rf.feature_importances_), columns=['Predictors', 'Feature Importances']).sort('Feature Importances', ascending=False)

R2 Score:  0.32390779247
Explained Variance Score:  0.323907969289
MAE:  0.0602570169812
MSE:  0.00614812221323
Median AE:  0.0472754240189

CPU times: user 7min 27s, sys: 4.66 s, total: 7min 32s
Wall time: 2min 42s


/Users/x/anaconda/envs/data/lib/python2.7/site-packages/ipykernel/__main__.py:51: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
